In [2]:
import numpy as np
import pandas as pd
from gekko import GEKKO
from itertools import product 

In [3]:
# Fill in coal specification
supplier = ['Banpu', 'Logplus', 'Mac', 'Tiger', 'AVRA', 'Spot2', 'Spot3', 'Spot4', 'Spot5']

# Coal specification
total_moisture = [28.66, 17.81, 22.51, 20.82, 24.74] + [23.00]*4
inherent_moisture = [11.40, 12.70, 12.91, 12.81, 15.31] + [10.00]*4

ad_basis = [True]*5 + [False]*4
ash_content = [4.96, 5.98, 6.80, 4.27, 4.83] + [6.00]*4
volatile_matter = [42.98, 40.42, 40.78, 41.43, 39.92] + [36.00]*4
fixed_carbon = [40.65, 40.90, 39.51, 41.49, 39.94] + [37.00]*4
sulphur_content = [0.41, 0.49, 0.45, 0.16, 0.32] + [0.50]*4
gross_calorific_value = [5918, 6167.25, 5794, 5962, 5634] + [5400, 5300, 5600, 5500]

# Ash Analysis
SiO2 = [45.42, 35.80, 42.80, 28.36, 47.30] + [40.82]*4
Al2O3 = [14.18, 20.63, 17.32, 17.38, 17.60] + [13.57]*4
Fe2O3 = [14.58, 10.14, 11.41, 9.79, 11.20] + [11.22]*4
CaO = [7.43, 16.31, 9.23, 19.68, 9.00] + [16.09]*4
MgO = [5.65, 3.88, 3.31, 5.32, 4.97] + [2.53]*4
Na2O = [0.99, 1.81, 1.39, 8.70, 0.68] + [2.24]*4
K2O = [1.24, 1.32, 1.24, 1.03, 1.10] + [0.76]*4
TiO2 = [0.56, 0.79, 0.71, 0.70, 0.92] + [0.55]*4
Mn3O4 = [0.24, 0.08, 0.14, 0.04, 0.08] + [0.06]*4
SO3 = [7.49, 8.46, 11.92, 6.70, 0.61] + [11.05]*4
P2O5 = [0.16, 0.25, 0.43, 0.51, 0.45] + [0.16]*4

df_coal_spec = pd.DataFrame({
    'Supplier': supplier,
    'Total Moisture (%)': total_moisture,
    'Inherent Moisture (%)': inherent_moisture,
})
df_coal_spec.set_index('Supplier', inplace=True)

AR_conversion_ratio = (100 - df_coal_spec['Total Moisture (%)']) / (100 - df_coal_spec['Inherent Moisture (%)'])
for i in range(len(df_coal_spec)):
    ratio = AR_conversion_ratio[i] if ad_basis[i] else 1
    df_coal_spec.loc[df_coal_spec.index[i], ['Ash Content (%)']] = ash_content[i] * ratio
    df_coal_spec.loc[df_coal_spec.index[i], ['Volatile Matter (%)']] = volatile_matter[i] * ratio
    df_coal_spec.loc[df_coal_spec.index[i], ['Fixed Carbon (%)']] = fixed_carbon[i] * ratio
    df_coal_spec.loc[df_coal_spec.index[i], ['Sulphur Content (%)']] = sulphur_content[i] * ratio
    df_coal_spec.loc[df_coal_spec.index[i], ['Gross Calorific Value (kcal/kg)']] = gross_calorific_value[i] * ratio

df_coal_spec['SiO2'] = SiO2
df_coal_spec['Al2O3'] = Al2O3
df_coal_spec['Fe2O3'] = Fe2O3
df_coal_spec['CaO'] = CaO
df_coal_spec['MgO'] = MgO
df_coal_spec['Na2O'] = Na2O
df_coal_spec['K2O'] = K2O
df_coal_spec['TiO2'] = TiO2
df_coal_spec['Mn3O4'] = Mn3O4
df_coal_spec['SO3'] = SO3
df_coal_spec['P2O5'] = P2O5

df_coal_spec['%S dry'] = df_coal_spec['Sulphur Content (%)'] * 100/(100 - df_coal_spec['Total Moisture (%)'])
df_coal_spec['B/A'] = df_coal_spec[['Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O']].sum(axis=1) / df_coal_spec[['SiO2', 'Al2O3', 'TiO2']].sum(axis=1)
df_coal_spec['Slacking Index'] = df_coal_spec['%S dry'] * df_coal_spec['B/A']
df_coal_spec['Fouling Index'] = df_coal_spec['Na2O'] * df_coal_spec['B/A']

date_start = '2020-06-16'
date_stop = '2020-07-31'

# ['Banpu', 'Logplus', 'Mac', 'Tiger', 'AVRA', 'Spot2', 'Spot3', 'Spot4', 'Spot5']
initial_stock = [30811, 36262, 0, 7000, 33158, 0, 0, 0, 0]
dead_stock = 10159.7833420158 + 42402

# Case Oct
incoming_stock = [
    ['2020-06-17', 'Tiger', 20000],
    ['2020-06-18', 'Tiger', 20000],
    ['2020-06-19', 'Tiger', 15000],
    ['2020-07-03', 'Banpu', 54000],
    ['2020-07-27', 'Logplus', 53000],
    ['2020-08-10', 'Tiger', 53000],
    ['2020-08-27', 'Mac', 53000],
    ['2020-09-11', 'Logplus', 53000],
    ['2020-09-24', 'Banpu', 52000],
    ['2020-10-07', 'Logplus', 53000],
    ['2020-10-23', 'Banpu', 52000],
    ['2020-11-09', 'Tiger', 53000],
    ['2020-11-22', 'Banpu', 52000],
    ['2020-12-08', 'Spot2', 52000],
    ['2020-12-24', 'Spot3', 52000],
    ['2021-01-09', 'Banpu', 52000],
    ['2021-01-24', 'Logplus', 52000],
]

coal_adjust_price = [3267, 1499, 2373, 2179, 2500, 2600, 2700, 2800, 2900]

df_coal_spec.T

Supplier,Banpu,Logplus,Mac,Tiger,AVRA,Spot2,Spot3,Spot4,Spot5
Total Moisture (%),28.66,17.81,22.51,20.82,24.74,23.00,23.00,23.00,23.00
Inherent Moisture (%),11.40,12.70,12.91,12.81,15.31,10.00,10.00,10.00,10.00
Ash Content (%),3.99,5.63,6.05,3.88,4.29,6.00,6.00,6.00,6.00
Volatile Matter (%),34.61,38.05,36.28,37.62,35.48,36.00,36.00,36.00,36.00
Fixed Carbon (%),32.73,38.51,35.15,37.68,35.49,37.00,37.00,37.00,37.00
Sulphur Content (%),0.33,0.46,0.40,0.15,0.28,0.50,0.50,0.50,0.50
Gross Calorific Value (kcal/kg),"4,765.13","5,806.26","5,155.32","5,414.28","5,006.67","5,400.00","5,300.00","5,600.00","5,500.00"
SiO2,45.42,35.80,42.80,28.36,47.30,40.82,40.82,40.82,40.82
Al2O3,14.18,20.63,17.32,17.38,17.60,13.57,13.57,13.57,13.57
Fe2O3,14.58,10.14,11.41,9.79,11.20,11.22,11.22,11.22,11.22


In [10]:
date_rng = pd.date_range(start=date_start, end=date_stop, freq='D')
date_rng = [d.strftime('%Y-%m-%d') for d in date_rng]

m = GEKKO()

### PARAMETERS
# coal_price = m.Param(coal_adjust_price)
# coal_in = [[m.Param(0) for d in date_rng] for s in supplier]
# coal_remain = [[m.Var(0) for d in date_rng] for s in supplier]
# coal_remain_total = [m.Var(0) for d in date_rng]

# set coal_in = incoming_stock data
# for d, s, n in incoming_stock:
#     if d in date_rng:
#         coal_in[supplier.index(s)][date_rng.index(d)].value = n

ratio_step = m.Param(10)
n_step = m.Intermediate(100 / ratio_step)

### VARIABLES
# Integer [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cfb12_ratio = [[m.Var(integer=True, lb=0, ub=n_step) for d in date_rng] for s in supplier]
cfb3_ratio = [[m.Var(integer=True, lb=0, ub=n_step) for d in date_rng] for s in supplier]

# cfb12_select = [[m.Var(integer=True, lb=0, ub=1) for d in date_rng] for s in supplier]
# cfb3_select = [[m.Var(integer=True, lb=0, ub=1) for d in date_rng] for s in supplier]

# spec_list = ['%S', '%Ash', 'GCV', 'SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O', 'TiO2', '%S dry', 'B/A', 'Slacking Index', 'Fouling Index']
# spec_list_lookup = ['Sulphur Content (%)', 'Ash Content (%)', 'Gross Calorific Value (kcal/kg)', 'SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O', 'TiO2', '%S dry', 'B/A', 'Slacking Index', 'Fouling Index']
# cfb12_blending = [[m.Var() for d in date_rng] for sp in spec_list]
# cfb3_blending = [[m.Var() for d in date_rng] for sp in spec_list]


### OBJECTIVE
# m.Obj(m.sum([cfb12_ratio[s][d] * coal_price[s] for s in range(len(supplier)) for d in range(len(date_rng))] + [cfb3_ratio[s][d] * coal_price[s] for s in range(len(supplier)) for d in range(len(date_rng))]))

### CONSTRAINTS
# total ratio = 100%, and selection not exceed 2
# for d in range(len(date_rng)):
#     if d == 0:
#         for s in range(len(supplier)):
#             m.Equation(cfb12_ratio[s][d] == 0)
#             m.Equation(cfb3_ratio[s][d] == 0)
#     else:
#         m.Equation(m.sum([cfb12_ratio[s][d] for s in range(len(supplier))]) == n_step)
#         m.Equation(m.sum([cfb3_ratio[s][d] for s in range(len(supplier))]) == n_step)

#         m.Equation(m.sum([m.if2(cfb12_ratio[s][d], 0, 1) for s in range(len(supplier))]) <= 2)
#         m.Equation(m.sum([m.if2(cfb3_ratio[s][d], 0, 1) for s in range(len(supplier))]) <= 2)

# if ratio > 0 then select = 1, else if ratio == 0 then select = 0
# for d in range(len(date_rng)):
#     for s in range(len(supplier)):
        # m.Equation(cfb12_ratio[s][d] <= n_step * cfb12_select[s][d])
        # m.Equation(cfb12_ratio[s][d] >= cfb12_select[s][d])
        # m.Equation(cfb3_ratio[s][d] <= n_step * cfb3_select[s][d])
        # m.Equation(cfb3_ratio[s][d] >= cfb3_select[s][d])

# total number of select not exceed 2
# for d in range(len(date_rng)):
#     if d > 0:
#         m.Equation(m.sum([cfb12_select[s][d] for s in range(len(supplier))]) <= 2)
#         m.Equation(m.sum([cfb3_select[s][d] for s in range(len(supplier))]) <= 2)

# # calculate coal blending spec
# for sp, d in product(range(len(spec_list)), range(len(date_rng))):
#     pass


    # if sp == spec_list.index('B/A'):
    #     model += xsum(cfb12_blending[spec_list.index(B)][d] for B in ['Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O']) >= xsum(cfb12_blending[spec_list.index(A)][d] for A in ['SiO2', 'Al2O3', 'TiO2'])
    #     model += cfb3_blending[sp][d] == xsum(cfb3_blending[spec_list.index(B)][d] for B in ['Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O']) / xsum(cfb3_blending[spec_list.index(A)][d] for A in ['SiO2', 'Al2O3', 'TiO2'])
    # elif sp == spec_list.index('Slacking Index'):
    #     model += cfb12_blending[sp][d] == cfb12_blending[spec_list.index('%S dry')][d] * cfb12_blending[spec_list.index('B/A')][d]
    #     model += cfb3_blending[sp][d] == cfb3_blending[spec_list.index('%S dry')][d] * cfb3_blending[spec_list.index('B/A')][d]
    # elif sp == spec_list.index('Fouling Index'):
    #     model += cfb12_blending[sp][d] == cfb12_blending[spec_list.index('Na2O')][d] * cfb12_blending[spec_list.index('B/A')][d]
    #     model += cfb3_blending[sp][d] == cfb3_blending[spec_list.index('Na2O')][d] * cfb3_blending[spec_list.index('B/A')][d]
    # else:
    #     model += cfb12_blending[sp][d] == xsum(cfb12_ratio[s][d] * df_coal_spec.loc[supplier[s], spec_list_lookup[sp]] for s in range(len(supplier))) / n_step
    #     model += cfb3_blending[sp][d] == xsum(cfb3_ratio[s][d] * df_coal_spec.loc[supplier[s], spec_list_lookup[sp]] for s in range(len(supplier))) / n_step

# calculate coal remain
# for s, d in product(range(len(supplier)), range(len(date_rng))):
#     if d == 0:
#         coal_remain[s][d].value = initial_stock[s]
#     else:
#         coal_remain[s][d].value = coal_remain[s][d - 1] + coal_in[s][d]

# calculate coal remain total
# for d in range(len(date_rng)):
#     coal_remain_total[d].value = m.sum([coal_remain[s][d] for s in range(len(supplier))]) + dead_stock

    
### OPTIMIZING
m.options.SOLVER = 1
m.solve()
# m.solve(disp=False)


### RESULTS
df = pd.DataFrame({'Date': date_rng})
# df.loc[:, ['Coal_In_{}'.format(s) for s in supplier]] = [[coal_in[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['Coal_Remain_{}'.format(s) for s in supplier]] = [[coal_remain[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['Coal_Remain_Total']] = [coal_remain_total[d].value[0] for d in range(len(df))]

df.loc[:, ['CFB12_Ratio_{}'.format(s) for s in supplier]] = [[cfb12_ratio[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['CFB12_Ratio_{}'.format(s) for s in supplier]] = [[ratio_step * cfb12_ratio[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]


# df.loc[:, ['CFB12_Select_{}'.format(s) for s in supplier]] = [[cfb12_select[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['CFB12_Blending_{}'.format(s) for s in spec_list]] = [[cfb12_blending[s][d].value[0] for s in range(len(spec_list))] for d in range(len(df))]
# df.loc[:, ['CFB3_Ratio_{}'.format(s) for s in supplier]] = [[ratio_step * cfb3_ratio[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['CFB3_Select_{}'.format(s) for s in supplier]] = [[cfb3_select[s][d].value[0] for s in range(len(supplier))] for d in range(len(df))]
# df.loc[:, ['CFB3_Blending_{}'.format(s) for s in spec_list]] = [[cfb3_blending[s][d].value[0] for s in range(len(spec_list))] for d in range(len(df))]

df.set_index('Date', inplace=True)
df.columns = pd.MultiIndex.from_tuples([(col[:col.rfind('_')], col[col.rfind('_') + 1:]) for col in df.columns])

# df = df.loc[:, (df != 0).any(axis=0)]   #hide zero columns
df.head(25)

apm 184.82.200.118_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          829
   Intermediates:            1
   Connections  :            0
   Equations    :            1
   Residuals    :            0
 
 Number of state variables:            828
 Number of total equations: -            0
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            828
 
 Number of bound variables: -          828
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    1 Dpth:    0 Lvs:    0 Obj:  0.00E+00 Gap:  0

CFB12_Ratio                                                
                 Banpu Logplus  Mac Tiger AVRA Spot2 Spot3 Spot4 Spot5
Date                                                                  
2020-06-16        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-17        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-18        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-19        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-20        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-21        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-22        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-23        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-24        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-25        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-26        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-27        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-28        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-29        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-06-30        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-01        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-02        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-03        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-04        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-05        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-06        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-07        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-08        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-09        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00
2020-07-10        0.00    0.00 0.00  0.00 0.00  0.00  0.00  0.00  0.00